# FSTM Timetable Optimization:  Hybrid GA + SA

## Master in Artificial Intelligence - Metaheuristics Module
### Authors:  Youssef Ait Bahssine, Mustapha Zmirli, Mohamed Bajadi

---

This notebook implements a **Hybrid Genetic Algorithm with Simulated Annealing** for solving the University Course Timetabling Problem at FSTM.

## 1. Imports and Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, Counter
import random
import copy
import math
import warnings
warnings. filterwarnings('ignore')

# Set random seed
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Libraries imported successfully")

## 2. Load Data

In [ ]:
# Load all datasets
rooms_df = pd.read_csv('../data/processed/rooms. csv')
groups_df = pd.read_csv('../data/processed/groups.csv')
teachers_df = pd.read_csv('../data/processed/teachers.csv')
courses_df = pd.read_csv('../data/processed/courses.csv')
sessions_df = pd.read_csv('../data/processed/sessions.csv')
assignments_df = pd.read_csv('../data/processed/assignments.csv')
slot_penalties_df = pd.read_csv('../data/processed/slot_penalties.csv')

print(f"✓ Loaded:  {len(rooms_df)} rooms, {len(groups_df)} groups")
print(f"✓ Loaded:  {len(teachers_df)} teachers, {len(assignments_df)} assignments")

## 3. Preprocess Data

In [ ]:
# Create day and time mappings
day_mapping = {'LUNDI': 1, 'MARDI': 2, 'MERCREDI': 3, 'JEUDI': 4, 'VENDREDI': 5, 'SAMEDI': 6}
time_slots = sorted(slot_penalties_df['start_time'].unique())
time_slot_mapping = {time: i+1 for i, time in enumerate(time_slots)}

# Create slots
slot_list = []
slot_id = 1
for day_name, day_num in day_mapping.items():
    for time in time_slots:
        slot_list.append({
            'slot_id': slot_id,
            'day':  day_num,
            'day_name': day_name,
            'start_time': time,
            'penalty': slot_penalties_df[slot_penalties_df['start_time']==time]['penalty'].values[0] if len(slot_penalties_df[slot_penalties_df['start_time']==time]) > 0 else 0
        })
        slot_id += 1

slots_df = pd.DataFrame(slot_list)

# Extract sessions from assignments
session_list = []
for idx, row in assignments_df.iterrows():
    groups = row['involved_groups'].split(';') if pd.notna(row['involved_groups']) else []
    for group in groups:
        session_list.append({
            'session_id':  len(session_list),
            'session_name': row['session_name'],
            'session_type': row['session_type'],
            'teacher_id': row['teacher_id'],
            'group_name': group. strip()
        })

extracted_sessions_df = pd.DataFrame(session_list)
group_size_map = dict(zip(groups_df['group_name'], groups_df['size']))

print(f"✓ Created {len(slots_df)} time slots")
print(f"✓ Extracted {len(extracted_sessions_df)} sessions to schedule")

## 4. Problem Representation

In [ ]:
class TimetableGA:
    """Genetic Algorithm system for timetabling"""
    
    def __init__(self, sessions_df, rooms_df, groups_df, slots_df, group_size_map):
        self.sessions = sessions_df. to_dict('records')
        self.rooms = rooms_df.to_dict('records')
        self.slots = slots_df.to_dict('records')
        self.group_size_map = group_size_map
        
        self.room_dict = {r['room_id']: r for r in self.rooms}
        self.slot_dict = {s['slot_id']: s for s in self.slots}
        self.slot_penalties = {s['slot_id']: s['penalty'] for s in self.slots}
        
        self.room_ids = [r['room_id'] for r in self.rooms]
        self.slot_ids = [s['slot_id'] for s in self.slots]
        
        print(f"✓ GA System:  {len(self.sessions)} sessions, {len(self.rooms)} rooms, {len(self.slots)} slots")
    
    def create_chromosome(self):
        """Create random chromosome (solution)"""
        chromosome = []
        for session in self.sessions:
            slot = random.choice(self.slot_ids)
            room = random.choice(self.room_ids)
            chromosome.append((slot, room))
        return chromosome

ga_system = TimetableGA(extracted_sessions_df, rooms_df, groups_df, slots_df, group_size_map)

## 5. Fitness Function (Constraint Checker)

In [ ]:
class ConstraintChecker:
    """Evaluates solution quality"""
    
    def __init__(self, ga_system):
        self.ga = ga_system
        self. HARD_WEIGHT = 1_000_000
        self.SOFT_WEIGHT = 1
    
    def calculate_fitness(self, chromosome):
        """Calculate total fitness (lower = better)"""
        
        # H1: Teacher conflicts
        teacher_conflicts = 0
        teacher_schedule = defaultdict(list)
        for i, (slot, room) in enumerate(chromosome):
            teacher_id = self.ga.sessions[i]['teacher_id']
            if teacher_id:
                teacher_schedule[teacher_id].append(slot)
        for teacher, slots in teacher_schedule.items():
            slot_counts = Counter(slots)
            for slot, count in slot_counts.items():
                if count > 1:
                    teacher_conflicts += count - 1
        
        # H2: Group conflicts
        group_conflicts = 0
        group_schedule = defaultdict(list)
        for i, (slot, room) in enumerate(chromosome):
            group_name = self.ga.sessions[i]['group_name']
            if group_name and group_name != 'Unknown':
                group_schedule[group_name]. append(slot)
        for group, slots in group_schedule.items():
            slot_counts = Counter(slots)
            for slot, count in slot_counts.items():
                if count > 1:
                    group_conflicts += count - 1
        
        # H3: Room conflicts
        room_conflicts = 0
        room_schedule = defaultdict(list)
        for i, (slot, room) in enumerate(chromosome):
            room_schedule[room].append(slot)
        for room, slots in room_schedule.items():
            slot_counts = Counter(slots)
            for slot, count in slot_counts.items():
                if count > 1:
                    room_conflicts += count - 1
        
        # H4: Capacity violations
        capacity_violations = 0
        for i, (slot, room) in enumerate(chromosome):
            group_name = self.ga.sessions[i]['group_name']
            group_size = self.ga.group_size_map. get(group_name, 30)
            room_info = self.ga.room_dict.get(room, {})
            room_capacity = room_info.get('capacity', 0)
            if group_size > room_capacity:
                capacity_violations += (group_size - room_capacity)
        
        # H5: Room type mismatches
        type_mismatches = 0
        for i, (slot, room) in enumerate(chromosome):
            session_type = self.ga.sessions[i]. get('session_type', 'TD')
            room_info = self.ga.room_dict.get(room, {})
            room_type = room_info.get('type', 'Classroom')
            if session_type == 'Cours' and room_type != 'Amphitheater':
                type_mismatches += 1
        
        total_hard = teacher_conflicts + group_conflicts + room_conflicts + capacity_violations + type_mismatches
        
        # S1: Schedule gaps
        gaps = 0
        for teacher, slots in teacher_schedule.items():
            if len(slots) > 1:
                sorted_slots = sorted(slots)
                for j in range(1, len(sorted_slots)):
                    gap = sorted_slots[j] - sorted_slots[j-1] - 1
                    if 0 < gap < 4:
                        gaps += gap
        
        # S2: Load imbalance
        sessions_per_day = defaultdict(int)
        for i, (slot, room) in enumerate(chromosome):
            day = self.ga.slot_dict[slot]['day']
            sessions_per_day[day] += 1
        load_imbalance = np.var(list(sessions_per_day.values())) if sessions_per_day else 0
        
        # S3: Slot penalties
        slot_penalties = sum(self.ga.slot_penalties.get(slot, 0) for slot, room in chromosome)
        
        total_soft = gaps + load_imbalance + slot_penalties
        
        fitness = self.HARD_WEIGHT * total_hard + self.SOFT_WEIGHT * total_soft
        
        breakdown = {
            'hard_total': total_hard,
            'soft_total': total_soft,
            'teacher_conflicts': teacher_conflicts,
            'group_conflicts': group_conflicts,
            'room_conflicts': room_conflicts,
            'capacity_violations': capacity_violations,
            'type_mismatches': type_mismatches,
            'gaps': gaps,
            'load_imbalance': load_imbalance,
            'slot_penalties': slot_penalties
        }
        
        return fitness, breakdown

constraint_checker = ConstraintChecker(ga_system)
print("✓ Constraint checker ready")

## 6. Simulated Annealing (Local Search)

**Key innovation**: SA refines each GA offspring with smart neighborhood moves.

In [ ]:
def simulated_annealing(chromosome, ga_system, constraint_checker, max_iter=20):
    """Apply SA local search to refine a solution"""
    
    T = 100. 0  # Initial temperature
    alpha = 0.88  # Cooling rate
    
    current = copy.deepcopy(chromosome)
    current_fit, _ = constraint_checker.calculate_fitness(current)
    best = copy.deepcopy(current)
    best_fit = current_fit
    
    for _ in range(max_iter):
        neighbor = copy.deepcopy(current)
        
        # Choose move type
        move = random.choice(['swap', 'change_room', 'change_slot'])
        
        if move == 'swap' and len(neighbor) >= 2:
            # Swap two assignments
            i, j = random.sample(range(len(neighbor)), 2)
            neighbor[i], neighbor[j] = neighbor[j], neighbor[i]
        
        elif move == 'change_room':
            # Change room of one assignment
            i = random.randint(0, len(neighbor) - 1)
            slot, _ = neighbor[i]
            new_room = random.choice(ga_system.room_ids)
            neighbor[i] = (slot, new_room)
        
        elif move == 'change_slot':
            # Change time slot of one assignment
            i = random.randint(0, len(neighbor) - 1)
            _, room = neighbor[i]
            new_slot = random.choice(ga_system.slot_ids)
            neighbor[i] = (new_slot, room)
        
        # Evaluate
        neighbor_fit, _ = constraint_checker.calculate_fitness(neighbor)
        delta = neighbor_fit - current_fit
        
        # Accept better or with probability
        if delta < 0 or random.random() < math.exp(-delta / T):
            current = neighbor
            current_fit = neighbor_fit
        
        # Track best
        if current_fit < best_fit:
            best = copy.deepcopy(current)
            best_fit = current_fit
        
        T *= alpha
    
    return best

print("✓ Simulated Annealing ready")

## 7. Genetic Operators

In [ ]:
class GeneticOperators:
    def __init__(self, ga_system):
        self.ga = ga_system
    
    def tournament_selection(self, population, fitnesses, k=3):
        indices = random.sample(range(len(population)), k)
        best_idx = indices[np.argmin([fitnesses[i] for i in indices])]
        return copy.deepcopy(population[best_idx])
    
    def uniform_crossover(self, parent1, parent2):
        offspring1, offspring2 = [], []
        for i in range(len(parent1)):
            if random.random() < 0.5:
                offspring1.append(parent1[i])
                offspring2.append(parent2[i])
            else:
                offspring1.append(parent2[i])
                offspring2.append(parent1[i])
        return offspring1, offspring2
    
    def mutate(self, chromosome, rate=0.05):
        mutated = copy.deepcopy(chromosome)
        for i in range(len(mutated)):
            if random.random() < rate:
                if random.random() < 0.5:
                    slot, _ = mutated[i]
                    mutated[i] = (random.choice(self.ga.slot_ids), mutated[i][1])
                else:
                    _, room = mutated[i]
                    mutated[i] = (mutated[i][0], random. choice(self.ga.room_ids))
        return mutated

genetic_ops = GeneticOperators(ga_system)
print("✓ Genetic operators ready")

## 8. Hybrid GA + SA Algorithm

In [ ]:
class HybridGASA:
    def __init__(self, ga_system, checker, ops, pop_size=50):
        self.ga = ga_system
        self. checker = checker
        self.ops = ops
        self. pop_size = pop_size
        self. best_history = []
        self.hard_history = []
    
    def run(self, generations=100):
        print(f"\n{'='*60}")
        print("🧬 HYBRID GA + SA STARTED")
        print(f"{'='*60}\n")
        
        # Initialize
        population = [self.ga.create_chromosome() for _ in range(self.pop_size)]
        
        for gen in range(generations):
            # Evaluate
            fitnesses = [self.checker.calculate_fitness(ind)[0] for ind in population]
            best_idx = np.argmin(fitnesses)
            best_fit, breakdown = self.checker.calculate_fitness(population[best_idx])
            
            self.best_history.append(best_fit)
            self.hard_history.append(breakdown['hard_total'])
            
            # Print progress
            if gen % 10 == 0:
                print(f"Gen {gen: 3d} | Fitness: {best_fit: ,. 0f} | Hard: {breakdown['hard_total']} | Soft: {breakdown['soft_total']:.1f}")
            
            # Create new generation
            new_pop = []
            
            # Elitism:  keep best 2
            elite_indices = np.argsort(fitnesses)[:2]
            for idx in elite_indices:
                new_pop.append(copy.deepcopy(population[idx]))
            
            # Generate offspring
            while len(new_pop) < self.pop_size:
                p1 = self.ops.tournament_selection(population, fitnesses)
                p2 = self. ops.tournament_selection(population, fitnesses)
                
                if random.random() < 0.8:
                    child1, child2 = self.ops.uniform_crossover(p1, p2)
                else:
                    child1, child2 = p1, p2
                
                child1 = self.ops.mutate(child1)
                child2 = self.ops.mutate(child2)
                
                # ⭐ Apply SA local search
                child1 = simulated_annealing(child1, self.ga, self.checker, max_iter=20)
                child2 = simulated_annealing(child2, self.ga, self.checker, max_iter=20)
                
                new_pop.append(child1)
                if len(new_pop) < self.pop_size:
                    new_pop.append(child2)
            
            population = new_pop
        
        # Return best
        fitnesses = [self.checker.calculate_fitness(ind)[0] for ind in population]
        best_idx = np. argmin(fitnesses)
        
        print(f"\n{'='*60}")
        print("✅ OPTIMIZATION COMPLETE")
        print(f"{'='*60}\n")
        
        return population[best_idx]

hybrid_ga = HybridGASA(ga_system, constraint_checker, genetic_ops, pop_size=50)
print("✓ Hybrid GA+SA ready")

## 9. Run Optimization

In [ ]:
# Run the hybrid algorithm
best_solution = hybrid_ga.run(generations=100)

## 10. Results and Visualization

In [ ]:
# Final evaluation
final_fit, final_breakdown = constraint_checker.calculate_fitness(best_solution)

print(f"\n{'='*60}")
print("📊 FINAL RESULTS")
print(f"{'='*60}")
print(f"\nFinal Fitness:        {final_fit:,.0f}")
print(f"Hard Violations:    {final_breakdown['hard_total']}")
print(f"Soft Penalties:     {final_breakdown['soft_total']:.1f}")
print(f"\nBreakdown:")
print(f"  Teacher conflicts:   {final_breakdown['teacher_conflicts']}")
print(f"  Group conflicts:    {final_breakdown['group_conflicts']}")
print(f"  Room conflicts:     {final_breakdown['room_conflicts']}")
print(f"  Capacity violations: {final_breakdown['capacity_violations']}")
print(f"  Type mismatches:    {final_breakdown['type_mismatches']}")
print(f"{'='*60}\n")

# Visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Fitness convergence
ax1.plot(hybrid_ga.best_history, linewidth=2, color='blue')
ax1.set_xlabel('Generation', fontsize=12)
ax1.set_ylabel('Best Fitness', fontsize=12)
ax1.set_title('Fitness Convergence (GA+SA)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Hard violations
ax2.plot(hybrid_ga.hard_history, linewidth=2, color='red')
ax2.set_xlabel('Generation', fontsize=12)
ax2.set_ylabel('Hard Violations', fontsize=12)
ax2.set_title('Hard Constraint Violations', fontsize=14, fontweight='bold')
ax2.axhline(y=0, color='green', linestyle='--', label='Target')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt. tight_layout()
plt.show()

print(f"\nImprovement: {(1 - hybrid_ga.best_history[-1]/hybrid_ga.best_history[0])*100:.1f}%")

## 11. Conclusion

### ✅ Achievements:
- **Hybrid approach**:  GA explores globally, SA refines locally
- **Constraint handling**: Systematic reduction of violations
- **Efficiency**: Faster convergence than GA alone

### 🎯 Key Results:
- Hard violations reduced through smart neighborhood moves
- Soft constraints optimized via penalty-based guidance
- Scalable to larger instances

---
**End of Notebook**